In [3]:
# Cell 1 - Install / imports + env load
# Fresh venv? Uncomment once:
# %pip install -r requirements.txt

import os
import json
import hashlib
import sqlite3
import re
from dataclasses import dataclass
from typing import Any, Dict, Optional, List, Tuple
from datetime import datetime, timezone

import requests
import pandas as pd
from dotenv import load_dotenv

load_dotenv()  # loads .env from the notebook working directory

EXA_API_KEY = (os.getenv("EXA_API_KEY") or "").strip()
EXA_SMOKE_NO_NETWORK = (os.getenv("EXA_SMOKE_NO_NETWORK") or "0").strip() == "1"

if not EXA_API_KEY and not EXA_SMOKE_NO_NETWORK:
    raise RuntimeError(
        "Missing EXA_API_KEY. Create .env from .env.example and set EXA_API_KEY=...\n"
        "For local smoke runs without network/API billing, set EXA_SMOKE_NO_NETWORK=1."
    )

if EXA_SMOKE_NO_NETWORK:
    print("Smoke mode enabled (no network): using local mock Exa responses.")
else:
    print("Loaded EXA_API_KEY from .env (not printing it).")


Loaded EXA_API_KEY from .env (not printing it).


In [4]:
# Cell 2 - Config

CONFIG = {
    # Exa search request settings
    "exa_endpoint": "https://api.exa.ai/search",
    "search_type": "auto",        # auto | neural | fast | deep
    "category": "people",         # keep focused on professional people profiles
    "num_results": 5,               # cheap default
    "user_location": "US",

    # Contents toggles (each enabled content type can add per-result cost)
    "use_text": False,              # expensive; leave off for first pass
    "use_highlights": True,         # recommended cheap default
    "highlights_per_url": 1,
    "highlight_num_sentences": 2,
    "use_summary": False,           # keep off unless needed

    # Optional domain controls (leave empty unless you want stricter scoping)
    "include_domains": [],
    "exclude_domains": [],

    # Safety / moderation
    "moderation": True,
    "redact_emails_phones": True,

    # Budget (hard stop for uncached calls only)
    "budget_cap_usd": 7.50,

    # Cache (prevents re-billing on repeat runs)
    "sqlite_path": "exa_cache.sqlite",
    "cache_ttl_hours": 24 * 30,  # 30 days
}

# Pricing assumptions (keep aligned with Exa pricing page before production use)
PRICING = {
    # Search request pricing tier by requested result count
    "search_1_25": 0.005,
    "search_26_100": 0.025,

    # Content extraction pricing (per page, per content type)
    "content_text_per_page": 0.001,
    "content_highlights_per_page": 0.001,
    "content_summary_per_page": 0.001,
}

print("CONFIG loaded")
print(json.dumps(CONFIG, indent=2))

CONFIG loaded
{
  "exa_endpoint": "https://api.exa.ai/search",
  "search_type": "auto",
  "category": "people",
  "num_results": 5,
  "user_location": "US",
  "use_text": false,
  "use_highlights": true,
  "highlights_per_url": 1,
  "highlight_num_sentences": 2,
  "use_summary": false,
  "include_domains": [],
  "exclude_domains": [],
  "moderation": true,
  "redact_emails_phones": true,
  "budget_cap_usd": 7.5,
  "sqlite_path": "exa_cache.sqlite",
  "cache_ttl_hours": 720
}


In [5]:
# Cell 3 - Exa call wrapper

def _canonical_json(obj: Any) -> str:
    return json.dumps(obj, sort_keys=True, ensure_ascii=False, separators=(",", ":"))


def _sha256(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()


def _estimate_cost_from_pricing(payload: Dict[str, Any], num_results: int) -> float:
    search_cost = PRICING["search_1_25"] if num_results <= 25 else PRICING["search_26_100"]

    contents_cost = 0.0
    contents = payload.get("contents") or {}
    if contents.get("text") is True:
        contents_cost += num_results * PRICING["content_text_per_page"]
    if isinstance(contents.get("highlights"), dict):
        contents_cost += num_results * PRICING["content_highlights_per_page"]
    if isinstance(contents.get("summary"), dict):
        contents_cost += num_results * PRICING["content_summary_per_page"]

    return round(search_cost + contents_cost, 6)


EMAIL_RE = re.compile(r"\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b", re.IGNORECASE)
PHONE_RE = re.compile(r"(\+?\d[\d\-\s().]{7,}\d)")
STREETISH_RE = re.compile(
    r"\b\d{1,5}\s+[A-Za-z0-9.\-]+\s+(Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct|Circle|Cir|Way)\b",
    re.IGNORECASE,
)


def redact_text(s: Optional[str]) -> Optional[str]:
    if not s or not CONFIG.get("redact_emails_phones", True):
        return s
    s = EMAIL_RE.sub("[REDACTED_EMAIL]", s)
    s = PHONE_RE.sub("[REDACTED_PHONE]", s)
    s = STREETISH_RE.sub("[REDACTED_ADDRESS]", s)
    return s


def extract_preview(result: Dict[str, Any], max_chars: int = 280) -> str:
    highlights = result.get("highlights")
    if isinstance(highlights, list) and highlights:
        return redact_text(" | ".join(str(x) for x in highlights)) or ""
    text = result.get("text")
    if isinstance(text, str) and text:
        return (redact_text(text[:max_chars]) or "")
    summary = result.get("summary")
    if isinstance(summary, str):
        return (redact_text(summary[:max_chars]) or "")
    return ""


@dataclass
class ExaCallMeta:
    cache_hit: bool
    request_hash: str
    request_payload: Dict[str, Any]
    estimated_cost_usd: float
    actual_cost_usd: Optional[float]
    request_id: Optional[str]
    resolved_search_type: Optional[str]
    created_at_utc: str


def build_exa_payload(query: str, *, num_results: Optional[int] = None) -> Dict[str, Any]:
    num_results = int(num_results or CONFIG["num_results"])
    payload: Dict[str, Any] = {
        "query": query,
        "type": CONFIG["search_type"],
        "category": CONFIG["category"],
        "numResults": num_results,
        "userLocation": CONFIG["user_location"],
        "moderation": CONFIG["moderation"],
    }

    if CONFIG["include_domains"]:
        payload["includeDomains"] = CONFIG["include_domains"]
    if CONFIG["exclude_domains"]:
        payload["excludeDomains"] = CONFIG["exclude_domains"]

    contents: Dict[str, Any] = {}
    if CONFIG["use_text"]:
        contents["text"] = True
    if CONFIG["use_highlights"]:
        contents["highlights"] = {
            "highlightsPerUrl": CONFIG["highlights_per_url"],
            "numSentences": CONFIG["highlight_num_sentences"],
        }
    if CONFIG["use_summary"]:
        contents["summary"] = {
            "query": "Summarize the person's professional background and insurance/CAT relevance."
        }

    if contents:
        payload["contents"] = contents

    return payload


def exa_search_people(query: str, *, num_results: Optional[int] = None) -> Tuple[Dict[str, Any], ExaCallMeta]:
    payload = build_exa_payload(query, num_results=num_results)
    estimated_cost = _estimate_cost_from_pricing(payload, int(payload["numResults"]))

    response_json, cache_hit = cache_get_or_set(payload, estimated_cost)  # defined in Cell 4

    actual_cost = None
    if isinstance(response_json, dict):
        cost = response_json.get("costDollars")
        if isinstance(cost, dict) and isinstance(cost.get("total"), (int, float)):
            actual_cost = float(cost["total"])

    meta = ExaCallMeta(
        cache_hit=cache_hit,
        request_hash=_sha256(_canonical_json(payload)),
        request_payload=payload,
        estimated_cost_usd=estimated_cost,
        actual_cost_usd=actual_cost,
        request_id=response_json.get("requestId") if isinstance(response_json, dict) else None,
        resolved_search_type=response_json.get("resolvedSearchType") if isinstance(response_json, dict) else None,
        created_at_utc=datetime.now(timezone.utc).isoformat(),
    )
    return response_json, meta

In [6]:
# Cell 4 - Cache wrapper (sqlite) + budget enforcement

def _db() -> sqlite3.Connection:
    conn = sqlite3.connect(CONFIG["sqlite_path"])
    conn.execute(
        """
        CREATE TABLE IF NOT EXISTS exa_cache (
            request_hash TEXT PRIMARY KEY,
            request_json TEXT NOT NULL,
            response_json TEXT NOT NULL,
            estimated_cost_usd REAL NOT NULL,
            actual_cost_usd REAL,
            created_at_utc TEXT NOT NULL
        )
        """
    )
    conn.execute(
        """
        CREATE TABLE IF NOT EXISTS exa_ledger (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            request_hash TEXT NOT NULL,
            query TEXT NOT NULL,
            cache_hit INTEGER NOT NULL,
            estimated_cost_usd REAL NOT NULL,
            actual_cost_usd REAL,
            created_at_utc TEXT NOT NULL
        )
        """
    )
    conn.commit()
    return conn


def cache_lookup(request_hash: str) -> Optional[Dict[str, Any]]:
    conn = _db()
    try:
        row = conn.execute(
            "SELECT response_json, created_at_utc FROM exa_cache WHERE request_hash = ?",
            (request_hash,),
        ).fetchone()
    finally:
        conn.close()

    if not row:
        return None

    response_json, created_at_utc = row
    try:
        created = datetime.fromisoformat(str(created_at_utc).replace("Z", "+00:00"))
    except ValueError:
        return None

    age_hours = (datetime.now(timezone.utc) - created).total_seconds() / 3600
    if age_hours > float(CONFIG["cache_ttl_hours"]):
        return None

    return json.loads(response_json)


def cache_store(request_hash: str, request_payload: Dict[str, Any], response_json: Dict[str, Any], estimated_cost: float) -> None:
    actual_cost = None
    if isinstance(response_json, dict):
        cost = response_json.get("costDollars")
        if isinstance(cost, dict) and isinstance(cost.get("total"), (int, float)):
            actual_cost = float(cost["total"])

    conn = _db()
    try:
        conn.execute(
            "INSERT OR REPLACE INTO exa_cache (request_hash, request_json, response_json, estimated_cost_usd, actual_cost_usd, created_at_utc) VALUES (?, ?, ?, ?, ?, ?)",
            (
                request_hash,
                _canonical_json(request_payload),
                _canonical_json(response_json),
                float(estimated_cost),
                actual_cost,
                datetime.now(timezone.utc).isoformat(),
            ),
        )
        conn.commit()
    finally:
        conn.close()


def ledger_add(request_hash: str, query: str, cache_hit: bool, estimated_cost: float, actual_cost: Optional[float]) -> None:
    conn = _db()
    try:
        conn.execute(
            "INSERT INTO exa_ledger (request_hash, query, cache_hit, estimated_cost_usd, actual_cost_usd, created_at_utc) VALUES (?, ?, ?, ?, ?, ?)",
            (
                request_hash,
                query,
                1 if cache_hit else 0,
                float(estimated_cost),
                None if actual_cost is None else float(actual_cost),
                datetime.now(timezone.utc).isoformat(),
            ),
        )
        conn.commit()
    finally:
        conn.close()


def ledger_summary() -> pd.DataFrame:
    conn = _db()
    try:
        df = pd.read_sql_query("SELECT * FROM exa_ledger ORDER BY id ASC", conn)
    finally:
        conn.close()

    expected_cols = [
        "id",
        "request_hash",
        "query",
        "cache_hit",
        "estimated_cost_usd",
        "actual_cost_usd",
        "created_at_utc",
    ]
    if df.empty:
        return pd.DataFrame(columns=expected_cols)
    return df


def spend_so_far() -> Dict[str, float]:
    df = ledger_summary()
    if df.empty:
        return {
            "request_count": 0,
            "cache_hits": 0,
            "uncached_calls": 0,
            "spent_usd": 0.0,
            "avg_cost_per_uncached_query": 0.0,
        }

    billable = []
    for _, row in df.iterrows():
        if int(row["cache_hit"]) == 1:
            billable.append(0.0)
        elif pd.notna(row["actual_cost_usd"]):
            billable.append(float(row["actual_cost_usd"]))
        else:
            billable.append(float(row["estimated_cost_usd"]))

    df = df.copy()
    df["billable_cost_usd"] = billable
    uncached_mask = df["cache_hit"].astype(int) == 0
    uncached_count = int(uncached_mask.sum())
    avg_uncached = float(df.loc[uncached_mask, "billable_cost_usd"].mean()) if uncached_count else 0.0

    return {
        "request_count": int(len(df)),
        "cache_hits": int((df["cache_hit"].astype(int) == 1).sum()),
        "uncached_calls": uncached_count,
        "spent_usd": round(float(df["billable_cost_usd"].sum()), 6),
        "avg_cost_per_uncached_query": round(avg_uncached, 6),
    }


def enforce_budget(next_estimated_cost: float) -> None:
    metrics = spend_so_far()
    projected_spend = float(metrics["spent_usd"]) + float(next_estimated_cost)
    if projected_spend > float(CONFIG["budget_cap_usd"]):
        raise RuntimeError(
            "Budget cap exceeded before uncached Exa call.\n"
            f"Spent so far: ${metrics['spent_usd']:.4f}\n"
            f"Next call estimate: ${float(next_estimated_cost):.4f}\n"
            f"Cap: ${float(CONFIG['budget_cap_usd']):.2f}\n"
            "Lower num_results and/or disable text/summary, or rerun cached queries."
        )


def _mock_exa_response(payload: Dict[str, Any]) -> Dict[str, Any]:
    query = str(payload.get("query") or "")
    num_results = int(payload.get("numResults") or 5)
    contents = payload.get("contents") or {}
    wants_highlights = isinstance(contents.get("highlights"), dict)
    wants_text = contents.get("text") is True
    wants_summary = isinstance(contents.get("summary"), dict)

    slug = _sha256(query)[:8]
    results: List[Dict[str, Any]] = []
    for i in range(num_results):
        title = f"Mock Professional Result {i+1} - CAT loss / insurance expert"
        url = f"https://www.linkedin.com/in/mock-{slug}-{i+1}"
        item: Dict[str, Any] = {
            "id": f"mock-{slug}-{i+1}",
            "title": title,
            "url": url,
        }
        if wants_highlights:
            item["highlights"] = [
                f"Mock highlight for query: {query}. Public professional profile for insurance litigation and expert witness context."
            ]
            item["highlightScores"] = [0.99]
        if wants_text:
            item["text"] = (
                "Mock text body. Public/professional info only. No personal addresses or contact harvesting. "
                f"Query context: {query}."
            )
        if wants_summary:
            item["summary"] = "Mock summary: relevant professional background for insurance/CAT-loss workflow evaluation."
        results.append(item)

    return {
        "requestId": f"smoke-{slug}",
        "resolvedSearchType": str(payload.get("type") or "auto"),
        "results": results,
        "costDollars": {
            "search": 0.0,
            "contents": 0.0,
            "total": 0.0,
        },
        "_smokeMode": True,
    }


def exa_http_call(payload: Dict[str, Any]) -> Dict[str, Any]:
    if EXA_SMOKE_NO_NETWORK:
        return _mock_exa_response(payload)

    if not EXA_API_KEY:
        raise RuntimeError("Missing EXA_API_KEY for live Exa request.")

    headers = {
        "x-api-key": EXA_API_KEY,
        "Content-Type": "application/json",
    }
    response = requests.post(CONFIG["exa_endpoint"], headers=headers, json=payload, timeout=60)
    response.raise_for_status()
    return response.json()


def cache_get_or_set(payload: Dict[str, Any], estimated_cost: float) -> Tuple[Dict[str, Any], bool]:
    request_hash = _sha256(_canonical_json(payload))
    cached = cache_lookup(request_hash)
    if cached is not None:
        cached_cost = None
        if isinstance(cached, dict):
            cost = cached.get("costDollars")
            if isinstance(cost, dict) and isinstance(cost.get("total"), (int, float)):
                cached_cost = float(cost["total"])
        ledger_add(
            request_hash=request_hash,
            query=str(payload.get("query") or ""),
            cache_hit=True,
            estimated_cost=estimated_cost,
            actual_cost=cached_cost,
        )
        return cached, True

    enforce_budget(estimated_cost)
    response_json = exa_http_call(payload)
    cache_store(request_hash, payload, response_json, estimated_cost)

    actual_cost = None
    if isinstance(response_json, dict):
        cost = response_json.get("costDollars")
        if isinstance(cost, dict) and isinstance(cost.get("total"), (int, float)):
            actual_cost = float(cost["total"])

    ledger_add(
        request_hash=request_hash,
        query=str(payload.get("query") or ""),
        cache_hit=False,
        estimated_cost=estimated_cost,
        actual_cost=actual_cost,
    )
    return response_json, False


print("Cache ready:", CONFIG["sqlite_path"])
print("Ledger metrics:", spend_so_far())

Cache ready: exa_cache.sqlite
Ledger metrics: {'request_count': 0, 'cache_hits': 0, 'uncached_calls': 0, 'spent_usd': 0.0, 'avg_cost_per_uncached_query': 0.0}


In [8]:
# Cell 5 - Single query demo

demo_query = "Florida property insurance attorney hurricane Ian appraisal dispute site:linkedin.com"
resp_demo, meta_demo = exa_search_people(demo_query, num_results=CONFIG["num_results"])

print("Single-query demo")
print("  cache_hit:", meta_demo.cache_hit)
print("  estimated_cost_usd:", meta_demo.estimated_cost_usd)
print("  actual_cost_usd:", meta_demo.actual_cost_usd)
print("  request_id:", meta_demo.request_id)
print("  resolved_search_type:", meta_demo.resolved_search_type)

results_demo = resp_demo.get("results", []) if isinstance(resp_demo, dict) else []
rows_demo = []
for result in results_demo[: int(CONFIG["num_results"])]:
    rows_demo.append(
        {
            "title": redact_text(result.get("title")),
            "url": result.get("url"),
            "preview": extract_preview(result, max_chars=280),
            "highlightScores": result.get("highlightScores"),
        }
    )

df_demo = pd.DataFrame(rows_demo)
print("Rows returned:", len(df_demo))
df_demo

Single-query demo
  cache_hit: True
  estimated_cost_usd: 0.01
  actual_cost_usd: 0.01
  request_id: 4ba4681290c69b1e0269bdb256e75d88
  resolved_search_type: neural
Rows returned: 5


,title,url,preview,highlightScores
0,Amy Boggs | Florida Property Insurance Attorney,https://linkedin.com/in/amy-boggs,practice focuses on the following insurance c...,[0.75]
1,Steven Venook | 25 yrs Exp Disputed Insurance ...,https://linkedin.com/in/claimexpert,"IES: Water damage, Mold damage, Lightning Dama...",[0.51953125]
2,Derrick Martin | Property & Casualty Claims Pr...,https://linkedin.com/in/derrick-martin-1b981b146,"ed carrier in mediation, appraisal, and other ...",[0.734375]
3,Douglas Anderson | Independent Insurance Adjus...,https://linkedin.com/in/douglas-anderson-89a924ba,.2018 - Present - Dispute resolution appraisal...,[0.515625]
4,James C. Murphy | Licensed Public Adjuster (FL...,https://linkedin.com/in/jcmadjuster,"Specialize in hurricane, wind, water, fire, a...",[0.7578125]


In [9]:
# Cell 5.5 — Mini batch sanity check (5 high-signal queries)

SANITY_QUERIES = [
    "forensic engineer wind damage expert witness Florida property insurance site:linkedin.com",
    "building envelope consultant moisture intrusion expert witness Florida site:linkedin.com",
    "forensic accountant business interruption insurance claim expert witness site:linkedin.com",
    "meteorologist hurricane wind field expert witness litigation site:linkedin.com",
    "policyholder attorney bad faith property insurance Florida site:linkedin.com",
]

rows = []
for q in SANITY_QUERIES:
    resp, meta = exa_search_people(q, num_results=CONFIG["num_results"])
    results = resp.get("results", []) if isinstance(resp, dict) else []
    top = results[0] if results else {}

    rows.append({
        "query": q,
        "cache_hit": meta.cache_hit,
        "est_cost_usd": meta.estimated_cost_usd,
        "actual_cost_usd": meta.actual_cost_usd,
        "top_title": redact_text(top.get("title")),
        "top_url": top.get("url"),
        "top_preview": redact_text(extract_preview(top, max_chars=220)) if "extract_preview" in globals() else redact_text(str(top.get("highlights", ""))[:220]),
        "top_highlightScore": (top.get("highlightScores") or [None])[0] if isinstance(top.get("highlightScores"), list) else None,
    })

df_sanity = pd.DataFrame(rows)
print("Spend so far:", spend_so_far())
df_sanity

Spend so far: {'request_count': 7, 'cache_hits': 1, 'uncached_calls': 6, 'spent_usd': 0.06, 'avg_cost_per_uncached_query': 0.01}


,query,cache_hit,est_cost_usd,actual_cost_usd,top_title,top_url,top_preview,top_highlightScore
0,forensic engineer wind damage expert witness F...,False,0.01,0.01,David Cowheard | Forensic & Dispute Services P...,https://linkedin.com/in/david-cowheard-a14b2314,witness testimony in the areas of insurance c...,0.472656
1,building envelope consultant moisture intrusio...,False,0.01,0.01,Don Williams | Principal at Moisture Intrusion...,https://linkedin.com/in/don-williams-94272611,clients in preventing future water intrusion ...,0.558594
2,forensic accountant business interruption insu...,False,0.01,0.01,John W. Foley | Forensic Accountant & Expert W...,https://linkedin.com/in/john-w-foley-31038571,"a qualified expert witness, I’ve testified or...",0.458984
3,meteorologist hurricane wind field expert witn...,False,0.01,0.01,Craig Setzer | Chief Meteorologist Royal Carib...,https://linkedin.com/in/craig-setzer-ccm-03095...,igation experience in matters related to meteo...,0.566406
4,policyholder attorney bad faith property insur...,False,0.01,0.01,Corey Harris | Attorney at Merlin Law Group,https://linkedin.com/in/corey-harris-345b7981,trial attorney exclusively representing polic...,0.406250


In [10]:
# Cell 6 - Batch query test suite (insurance / CAT)

# Swap this list to evaluate other workflows while keeping the rest of the notebook unchanged.
TEST_QUERIES = [
    "forensic engineer wind damage expert witness Florida property insurance site:linkedin.com",
    "building envelope consultant moisture intrusion expert witness Florida site:linkedin.com",
    "forensic accountant business interruption insurance claim expert witness site:linkedin.com",
    "meteorologist hurricane wind field expert witness litigation site:linkedin.com",
    "fire origin and cause investigator expert witness insurance litigation site:linkedin.com",
    "policyholder attorney bad faith property insurance Florida site:linkedin.com",
    "Texas hail damage property insurance attorney appraisal dispute site:linkedin.com",
    "insurance appraisal umpire property claims Florida site:linkedin.com",
    "licensed public adjuster large loss hurricane Florida commercial property site:linkedin.com",
    "Xactimate trainer estimator large loss consultant site:linkedin.com",
    "claims consultant catastrophe response litigation support property insurance site:linkedin.com",
    "insurance coverage expert witness former adjuster property claims site:linkedin.com",
    "water mitigation IICRC expert witness insurance dispute site:linkedin.com",
    "roofing consultant wind uplift tile roof expert witness Florida site:linkedin.com",
    "civil engineer structural damage assessment hurricane expert witness site:linkedin.com",
]

RELEVANCE_KEYWORDS = [
    "expert witness",
    "forensic",
    "insurance",
    "appraisal",
    "adjuster",
    "coverage",
    "litigation",
    "catastrophe",
]

batch_rows = []
for query in TEST_QUERIES:
    resp, meta = exa_search_people(query, num_results=CONFIG["num_results"])
    results = resp.get("results", []) if isinstance(resp, dict) else []
    top = results[0] if results else {}

    top_n = results[: int(CONFIG["num_results"])]
    linkedin_present = any("linkedin.com" in str(r.get("url") or "").lower() for r in top_n)

    relevance_text = []
    for r in top_n:
        parts = [str(r.get("title") or "")]
        highlights = r.get("highlights")
        if isinstance(highlights, list):
            parts.extend(str(x) for x in highlights)
        text = r.get("text")
        if isinstance(text, str):
            parts.append(text[:200])
        relevance_text.append(" ".join(parts).lower())
    relevance_keywords_present = any(any(k in blob for k in RELEVANCE_KEYWORDS) for blob in relevance_text)

    batch_rows.append(
        {
            "query": query,
            "cache_hit": meta.cache_hit,
            "est_cost_usd": meta.estimated_cost_usd,
            "actual_cost_usd": meta.actual_cost_usd,
            "top_title": redact_text(top.get("title")) if isinstance(top, dict) else None,
            "top_url": top.get("url") if isinstance(top, dict) else None,
            "top_preview": extract_preview(top, max_chars=220) if isinstance(top, dict) else "",
            "linkedin_present": linkedin_present,
            "relevance_keywords_present": relevance_keywords_present,
            "result_count": len(results),
        }
    )

df_batch = pd.DataFrame(batch_rows)
print(f"Batch queries executed: {len(df_batch)}")
df_batch

Batch queries executed: 15


,query,cache_hit,est_cost_usd,actual_cost_usd,top_title,top_url,top_preview,linkedin_present,relevance_keywords_present,result_count
0,forensic engineer wind damage expert witness F...,True,0.01,0.01,David Cowheard | Forensic & Dispute Services P...,https://linkedin.com/in/david-cowheard-a14b2314,witness testimony in the areas of insurance c...,True,True,5
1,building envelope consultant moisture intrusio...,True,0.01,0.01,Don Williams | Principal at Moisture Intrusion...,https://linkedin.com/in/don-williams-94272611,clients in preventing future water intrusion ...,True,True,5
2,forensic accountant business interruption insu...,True,0.01,0.01,John W. Foley | Forensic Accountant & Expert W...,https://linkedin.com/in/john-w-foley-31038571,"a qualified expert witness, I’ve testified or...",True,True,5
3,meteorologist hurricane wind field expert witn...,True,0.01,0.01,Craig Setzer | Chief Meteorologist Royal Carib...,https://linkedin.com/in/craig-setzer-ccm-03095...,igation experience in matters related to meteo...,True,True,5
4,fire origin and cause investigator expert witn...,False,0.01,0.01,Michael Whedon | Fire and Explosion Investigator,https://linkedin.com/in/michael-whedon-85a639b5,"to insurance companies, private industry and ...",True,True,5
5,policyholder attorney bad faith property insur...,True,0.01,0.01,Corey Harris | Attorney at Merlin Law Group,https://linkedin.com/in/corey-harris-345b7981,trial attorney exclusively representing polic...,True,True,5
6,Texas hail damage property insurance attorney ...,False,0.01,0.01,David Rocha | Claims Examiner,https://linkedin.com/in/david-rocha-13997217,"settlement of wind and hail property losses, ...",True,True,5
7,insurance appraisal umpire property claims Flo...,False,0.01,0.01,"Michael Ruskin | Umpire, Insurance Appraiser, ...",https://linkedin.com/in/michael-ruskin-09214115,". Offices located in Florida, North Carolina, ...",True,True,5
8,licensed public adjuster large loss hurricane ...,False,0.01,0.01,Frank Altieri | The Policyholders’ Adjuster,https://linkedin.com/in/frank-altieri-5b7bb743,Altieri Insurance Consultants is a nationwide ...,True,True,5
9,Xactimate trainer estimator large loss consult...,False,0.01,0.01,Weston Faulkner | Large Loss Adjuster / Estima...,https://linkedin.com/in/weston-faulkner-6b4b5a1a,knowledge in Xactimate Estimating Software. T...,True,True,5


In [11]:
# Cell 6.5 — Compare: baseline (LinkedIn-biased) vs no-LinkedIn variant

import re
import pandas as pd

# If your Cell 6 dataframe is named differently, update this:
df_baseline = df_batch

def strip_linkedin_bias(q: str) -> str:
    # remove common patterns that force LinkedIn
    q2 = re.sub(r"\s+site:linkedin\.com\b", "", q, flags=re.IGNORECASE)
    q2 = re.sub(r"\s+site:\s*linkedin\.com\b", "", q2, flags=re.IGNORECASE)
    return q2.strip()

TEST_QUERIES_NO_LI = [strip_linkedin_bias(q) for q in TEST_QUERIES]

rows = []
for q in TEST_QUERIES_NO_LI:
    resp, meta = exa_search_people(q, num_results=CONFIG["num_results"])
    results = resp.get("results", []) if isinstance(resp, dict) else []
    top = results[0] if results else {}

    has_linkedin = any("linkedin.com" in (r.get("url") or "") for r in results[:CONFIG["num_results"]])

    # Keep the same lightweight relevance heuristic as Cell 6
    blob = ((top.get("title") or "") + " " + " ".join(top.get("highlights", []) if isinstance(top.get("highlights"), list) else [])).lower()
    relevance_hit = any(k in blob for k in ["expert witness", "forensic", "insurance", "appraisal", "adjuster", "coverage", "litigation"])

    rows.append({
        "query_no_li": q,
        "cache_hit": meta.cache_hit,
        "est_cost_usd": meta.estimated_cost_usd,
        "actual_cost_usd": meta.actual_cost_usd,
        "top_title": redact_text(top.get("title")),
        "top_url": top.get("url"),
        "top_preview": redact_text(extract_preview(top, max_chars=220)) if "extract_preview" in globals() else redact_text(str(top.get("highlights", ""))[:220]),
        "linkedin_present": has_linkedin,
        "relevance_keywords_present": relevance_hit,
        "result_count": len(results),
    })

df_no_li = pd.DataFrame(rows)

# Build a side-by-side comparison by row order (since it's the same list length/order)
compare = pd.DataFrame({
    "baseline_query": df_baseline["query"].tolist(),
    "no_li_query": df_no_li["query_no_li"].tolist(),

    "baseline_top_title": df_baseline["top_title"].tolist(),
    "no_li_top_title": df_no_li["top_title"].tolist(),

    "baseline_top_url": df_baseline["top_url"].tolist(),
    "no_li_top_url": df_no_li["top_url"].tolist(),

    "baseline_linkedin_present": df_baseline["linkedin_present"].tolist() if "linkedin_present" in df_baseline.columns else [None]*len(df_no_li),
    "no_li_linkedin_present": df_no_li["linkedin_present"].tolist(),

    "baseline_relevance_flag": df_baseline["relevance_keywords_present"].tolist() if "relevance_keywords_present" in df_baseline.columns else [None]*len(df_no_li),
    "no_li_relevance_flag": df_no_li["relevance_keywords_present"].tolist(),

    "baseline_actual_cost": df_baseline["actual_cost_usd"].tolist(),
    "no_li_actual_cost": df_no_li["actual_cost_usd"].tolist(),
})

print("=== Spend so far ===")
print(spend_so_far())

print("\n=== Quick deltas ===")
baseline_li_rate = float(pd.Series(compare["baseline_linkedin_present"]).mean()) if compare["baseline_linkedin_present"].notna().any() else None
no_li_li_rate = float(pd.Series(compare["no_li_linkedin_present"]).mean())

baseline_rel_rate = float(pd.Series(compare["baseline_relevance_flag"]).mean()) if compare["baseline_relevance_flag"].notna().any() else None
no_li_rel_rate = float(pd.Series(compare["no_li_relevance_flag"]).mean())

print("Baseline LinkedIn present rate:", baseline_li_rate)
print("No-LI LinkedIn present rate:", round(no_li_li_rate, 3))
print("Baseline relevance-flag rate:", baseline_rel_rate)
print("No-LI relevance-flag rate:", round(no_li_rel_rate, 3))

# Show the comparison table (titles + urls are usually enough to eyeball quality)
compare

=== Spend so far ===
{'request_count': 37, 'cache_hits': 6, 'uncached_calls': 31, 'spent_usd': 0.31, 'avg_cost_per_uncached_query': 0.01}

=== Quick deltas ===
Baseline LinkedIn present rate: 1.0
No-LI LinkedIn present rate: 1.0
Baseline relevance-flag rate: 1.0
No-LI relevance-flag rate: 0.933


,baseline_query,no_li_query,baseline_top_title,no_li_top_title,baseline_top_url,no_li_top_url,baseline_linkedin_present,no_li_linkedin_present,baseline_relevance_flag,no_li_relevance_flag,baseline_actual_cost,no_li_actual_cost
0,forensic engineer wind damage expert witness F...,forensic engineer wind damage expert witness F...,David Cowheard | Forensic & Dispute Services P...,Blake Shatto | Forensic Engineering Consultant...,https://linkedin.com/in/david-cowheard-a14b2314,https://linkedin.com/in/blake-shatto,True,True,True,True,0.01,0.01
1,building envelope consultant moisture intrusio...,building envelope consultant moisture intrusio...,Don Williams | Principal at Moisture Intrusion...,Don Williams | Principal at Moisture Intrusion...,https://linkedin.com/in/don-williams-94272611,https://linkedin.com/in/don-williams-94272611,True,True,True,True,0.01,0.01
2,forensic accountant business interruption insu...,forensic accountant business interruption insu...,John W. Foley | Forensic Accountant & Expert W...,Kevin Grudzien | Forensic Accountant | Claim P...,https://linkedin.com/in/john-w-foley-31038571,https://linkedin.com/in/kevin-grudzien-4712806,True,True,True,True,0.01,0.01
3,meteorologist hurricane wind field expert witn...,meteorologist hurricane wind field expert witn...,Craig Setzer | Chief Meteorologist Royal Carib...,Craig Setzer | Chief Meteorologist Royal Carib...,https://linkedin.com/in/craig-setzer-ccm-03095...,https://linkedin.com/in/craig-setzer-ccm-03095...,True,True,True,True,0.01,0.01
4,fire origin and cause investigator expert witn...,fire origin and cause investigator expert witn...,Michael Whedon | Fire and Explosion Investigator,IAAI-CFI Frank A. Valenti | Fire Investigation...,https://linkedin.com/in/michael-whedon-85a639b5,https://linkedin.com/in/frank-a-valenti-iaai-c...,True,True,True,True,0.01,0.01
5,policyholder attorney bad faith property insur...,policyholder attorney bad faith property insur...,Corey Harris | Attorney at Merlin Law Group,Corey Harris | Attorney at Merlin Law Group,https://linkedin.com/in/corey-harris-345b7981,https://linkedin.com/in/corey-harris-345b7981,True,True,True,True,0.01,0.01
6,Texas hail damage property insurance attorney ...,Texas hail damage property insurance attorney ...,David Rocha | Claims Examiner,Micah Harrison | Property Claims Professional,https://linkedin.com/in/david-rocha-13997217,https://linkedin.com/in/micah-harrison-956484112,True,True,True,False,0.01,0.01
7,insurance appraisal umpire property claims Flo...,insurance appraisal umpire property claims Flo...,"Michael Ruskin | Umpire, Insurance Appraiser, ...","Michael Ruskin | Umpire, Insurance Appraiser, ...",https://linkedin.com/in/michael-ruskin-09214115,https://linkedin.com/in/michael-ruskin-09214115,True,True,True,True,0.01,0.01
8,licensed public adjuster large loss hurricane ...,licensed public adjuster large loss hurricane ...,Frank Altieri | The Policyholders’ Adjuster,Frank Altieri | The Policyholders’ Adjuster,https://linkedin.com/in/frank-altieri-5b7bb743,https://linkedin.com/in/frank-altieri-5b7bb743,True,True,True,True,0.01,0.01
9,Xactimate trainer estimator large loss consult...,Xactimate trainer estimator large loss consultant,Weston Faulkner | Large Loss Adjuster / Estima...,Mark Hutcheson | Large Loss Consultant & Estim...,https://linkedin.com/in/weston-faulkner-6b4b5a1a,https://linkedin.com/in/mark-hutcheson-32063b64,True,True,True,True,0.01,0.01


In [14]:
# Cell 6.6 — Non-LinkedIn sources test (GENERAL search, LinkedIn excluded)

import pandas as pd
import re
from urllib.parse import urlparse

def _domain(url: str) -> str:
    try:
        return urlparse(url).netloc.lower()
    except Exception:
        return ""

def strip_site_linkedin(q: str) -> str:
    return re.sub(r"\s+site:linkedin\.com\b", "", q, flags=re.IGNORECASE).strip()

def exa_search_general(query: str, *, num_results: int = 5, exclude_domains=None, include_domains=None):
    exclude_domains = exclude_domains or []
    include_domains = include_domains or []

    payload = {
        "query": query,
        "type": CONFIG["search_type"],
        "numResults": int(num_results),
        "userLocation": CONFIG["user_location"],
        "moderation": CONFIG["moderation"],
    }

    # contents (same settings: highlights only)
    contents = {}
    if CONFIG["use_text"]:
        contents["text"] = True
    if CONFIG["use_highlights"]:
        contents["highlights"] = {
            "highlightsPerUrl": CONFIG["highlights_per_url"],
            "numSentences": CONFIG["highlight_num_sentences"],
        }
    if CONFIG["use_summary"]:
        contents["summary"] = {"query": "Summarize why this is relevant."}
    if contents:
        payload["contents"] = contents

    # ✅ LINKEDIN EXCLUSION IS HERE
    if include_domains:
        payload["includeDomains"] = include_domains
    if exclude_domains:
        payload["excludeDomains"] = exclude_domains

    est = _estimate_cost_from_pricing(payload, int(payload["numResults"]))
    resp, cache_hit = cache_get_or_set(payload, est)

    actual = None
    if isinstance(resp, dict):
        cd = resp.get("costDollars")
        if isinstance(cd, dict) and isinstance(cd.get("total"), (int, float)):
            actual = float(cd["total"])

    return resp, {"cache_hit": cache_hit, "estimated_cost_usd": est, "actual_cost_usd": actual}

# Build “no forced LinkedIn” query list from your existing TEST_QUERIES
TEST_QUERIES_GENERAL = [strip_site_linkedin(q) for q in TEST_QUERIES]

# ✅ Update this list to exclude ALL linkedin variants
EXCLUDE = ["linkedin.com", "www.linkedin.com", "lnkd.in"]

rows = []
for q in TEST_QUERIES_GENERAL:
    resp, meta = exa_search_general(q, num_results=5, exclude_domains=EXCLUDE)
    results = resp.get("results", []) if isinstance(resp, dict) else []
    top = results[0] if results else {}

    top_url = top.get("url") or ""
    top_dom = _domain(top_url)

    # preview
    if isinstance(top.get("highlights"), list) and top.get("highlights"):
        preview = " | ".join(top["highlights"])[:240]
    elif isinstance(top.get("text"), str) and top.get("text"):
        preview = top["text"][:240]
    else:
        preview = ""

    rows.append({
        "query": q,
        "cache_hit": meta["cache_hit"],
        "est_cost_usd": meta["estimated_cost_usd"],
        "actual_cost_usd": meta["actual_cost_usd"],
        "top_title": redact_text(top.get("title")),
        "top_url": top_url,
        "top_domain": top_dom,
        "top_preview": redact_text(preview),
        "linkedin_present_in_top": ("linkedin.com" in top_dom or "lnkd.in" in top_dom),
        "result_count": len(results),
    })

df_non_linkedin = pd.DataFrame(rows)

print("Spend so far:", spend_so_far())
print("Top-result LinkedIn rate:", round(float(df_non_linkedin["linkedin_present_in_top"].mean()), 3))
df_non_linkedin

Spend so far: {'request_count': 67, 'cache_hits': 6, 'uncached_calls': 61, 'spent_usd': 0.596, 'avg_cost_per_uncached_query': 0.00977}
Top-result LinkedIn rate: 0.533


,query,cache_hit,est_cost_usd,actual_cost_usd,top_title,top_url,top_domain,top_preview,linkedin_present_in_top,result_count
0,forensic engineer wind damage expert witness F...,False,0.01,0.010,"Raul E Rivero | Expert Witness, 35 year vetera...",https://linkedin.com/in/raul-e-rivero-mba-cgc-...,linkedin.com,"Expert Witness, 35 year veteran in the Insuran...",True,5
1,building envelope consultant moisture intrusio...,False,0.01,0.010,DALE E. RENNINGER | Flat Roof and/or Wall Cons...,https://linkedin.com/in/dale-e-renninger-crt-0...,linkedin.com,Roof & Wall Water Intrusion Expert. Invented ...,True,5
2,forensic accountant business interruption insu...,False,0.01,0.009,Drew Olson | Experienced Forensic Accountant s...,https://linkedin.com/in/insuranceclaims,linkedin.com,as a neutral umpire in disputes regarding com...,True,4
3,meteorologist hurricane wind field expert witn...,False,0.01,0.009,Meteorology Expert Witness | Forensic Weather ...,https://weatherandclimateexpert.com/,weatherandclimateexpert.com,John Bryant is a ... With over **25 years of e...,False,4
4,fire origin and cause investigator expert witn...,False,0.01,0.009,Fire Cause and Origin Expert Witnesses: Key Le...,https://www.forensisgroup.com/resources/expert...,www.forensisgroup.com,origin investigation expert witnesses play a p...,False,4
5,policyholder attorney bad faith property insur...,False,0.01,0.010,Chip Merlin | Merlin Law Group | Insurance Cla...,https://linkedin.com/in/chip-merlin-15aaa811,linkedin.com,"noted national expert on Insurance Bad Faith,...",True,5
6,Texas hail damage property insurance attorney ...,False,0.01,0.010,Hail Damage Claims and Insurance Disputes | Pa...,https://panditlaw.com/hail-damage-claims/,panditlaw.com,Claims are governed by the Texas Prompt Paymen...,False,5
7,insurance appraisal umpire property claims Flo...,False,0.01,0.009,Experienced Public Insurance Appraisers and Um...,https://claimsaidconsultants.com/appraiser-ump...,claimsaidconsultants.com,ers with the designations of Certified Propert...,False,4
8,licensed public adjuster large loss hurricane ...,False,0.01,0.007,Large Loss Insurance Claims,https://askanadjuster.com/types-of-claims/larg...,askanadjuster.com,Licensed Large Loss Adjusters come to your com...,False,2
9,Xactimate trainer estimator large loss consultant,False,0.01,0.009,Stephen P. Harmon XCT / GC / Building Consulta...,https://linkedin.com/in/stephen-p-harmon-xct-g...,linkedin.com,"er, Xactimate Trainer, Assitance War Room Team...",True,4


In [16]:
# Cell 6.7 — Credibility pass: request more results, then PICK first non-LinkedIn result

import pandas as pd
from urllib.parse import urlparse

def _host(url: str) -> str:
    try:
        return urlparse(url).netloc.lower()
    except Exception:
        return ""

def is_linkedin_url(url: str) -> bool:
    h = _host(url)
    return ("linkedin.com" in h) or ("lnkd.in" in h)

# Stronger LinkedIn exclusion patterns (server-side)
EXCLUDE = [
    "*.linkedin.com",
    "linkedin.com/in",
    "linkedin.com/pub",
    "linkedin.com/company",
    "linkedin.com/school",
    "linkedin.com/jobs",
    "lnkd.in",
]

# Bump results to 10 so we have room to skip LinkedIn
N = 10

rows = []
for q in TEST_QUERIES_GENERAL:  # from your Cell 6.6 (queries without forcing site:linkedin.com)
    resp, meta = exa_search_general(q, num_results=N, exclude_domains=EXCLUDE)
    results = resp.get("results", []) if isinstance(resp, dict) else []

    top = results[0] if results else {}
    # Client-side pick: first non-LinkedIn
    picked = None
    for r in results:
        if not is_linkedin_url(r.get("url") or ""):
            picked = r
            break
    if picked is None:
        picked = top  # fallback if everything is LinkedIn

    def preview(r):
        if isinstance(r.get("highlights"), list) and r.get("highlights"):
            return " | ".join(r["highlights"])[:240]
        if isinstance(r.get("text"), str) and r.get("text"):
            return r["text"][:240]
        return ""

    rows.append({
        "query": q,
        "cache_hit": meta["cache_hit"],
        "actual_cost_usd": meta["actual_cost_usd"],

        "top_domain": _host(top.get("url") or ""),
        "top_title": redact_text(top.get("title")),
        "top_url": top.get("url"),
        "top_is_linkedin": is_linkedin_url(top.get("url") or ""),

        "picked_domain": _host(picked.get("url") or ""),
        "picked_title": redact_text(picked.get("title")),
        "picked_url": picked.get("url"),
        "picked_is_linkedin": is_linkedin_url(picked.get("url") or ""),

        "picked_preview": redact_text(preview(picked)),
        "result_count": len(results),
    })

df_cred = pd.DataFrame(rows)

print("Spend so far:", spend_so_far())
print("Top-result LinkedIn rate:", round(float(df_cred["top_is_linkedin"].mean()), 3))
print("Picked-non-LinkedIn rate:", round(float(df_cred["picked_is_linkedin"].mean()), 3), "(lower is better)")
df_cred[["query","top_domain","top_is_linkedin","picked_domain","picked_is_linkedin","picked_title","picked_url","picked_preview","actual_cost_usd","result_count"]]

df_cred.to_csv("credibility_pass_results.csv", index=False)
print("Wrote credibility_pass_results.csv")

Spend so far: {'request_count': 97, 'cache_hits': 21, 'uncached_calls': 76, 'spent_usd': 0.805, 'avg_cost_per_uncached_query': 0.010592}
Top-result LinkedIn rate: 0.4
Picked-non-LinkedIn rate: 0.0 (lower is better)
Wrote credibility_pass_results.csv


In [18]:
# Cell 7 - Summary table + qualitative notes

df_ledger = ledger_summary()
summary = spend_so_far()

print(f"Request count: {summary['request_count']}")
print(f"Cache hits vs uncached calls: {summary['cache_hits']} vs {summary['uncached_calls']}")
print(f"Spent so far (USD): ${summary['spent_usd']:.4f}")
print(f"Avg cost per uncached query (USD): ${summary['avg_cost_per_uncached_query']:.4f}")

summary_table = pd.DataFrame([summary])
print("\nSummary table")
try:
    print(summary_table.to_markdown(index=False))
except Exception:
    print(summary_table)

qualitative_notes: List[str] = []
if not df_batch.empty:
    relevance_rate = float(df_batch["relevance_keywords_present"].mean())
    linkedin_rate = float(df_batch["linkedin_present"].mean())
    avg_results = float(df_batch["result_count"].mean())
    qualitative_notes.append(f"Observed relevance-keyword signal rate: {relevance_rate:.0%} across batch queries.")
    qualitative_notes.append(f"LinkedIn/public professional profile signal appeared in {linkedin_rate:.0%} of queries.")
    qualitative_notes.append(f"Average result count returned per query: {avg_results:.1f} (configured num_results={CONFIG['num_results']}).")
else:
    qualitative_notes.append("No batch results yet. Run Cell 6 first.")

if CONFIG["use_text"]:
    qualitative_notes.append("Text is enabled: higher evidence quality, higher cost. Consider disabling for initial screening.")
else:
    qualitative_notes.append("Text is disabled: cheaper baseline. Highlights should usually be enough for triage.")

if CONFIG["use_summary"]:
    qualitative_notes.append("Summary is enabled: validate value before scaling because it adds per-result cost.")
else:
    qualitative_notes.append("Summary is disabled (recommended for cost-sensitive baseline testing).")

if EXA_SMOKE_NO_NETWORK:
    qualitative_notes.append("Smoke mode is on: results are mocked and costs are zero; use a real API key for live quality/cost evaluation.")

print("\nQualitative notes")
for note in qualitative_notes:
    print("-", note)

review_cols = [
    "query",
    "top_title",
    "top_url",
    "top_preview",
    "linkedin_present",
    "relevance_keywords_present",
    "cache_hit",
    "actual_cost_usd",
    "est_cost_usd",
]
df_batch[review_cols]


Request count: 97
Cache hits vs uncached calls: 21 vs 76
Spent so far (USD): $0.8050
Avg cost per uncached query (USD): $0.0106

Summary table
   request_count  cache_hits  uncached_calls  spent_usd  \
0             97          21              76      0.805   

   avg_cost_per_uncached_query  
0                     0.010592  

Qualitative notes
- Observed relevance-keyword signal rate: 100% across batch queries.
- LinkedIn/public professional profile signal appeared in 100% of queries.
- Average result count returned per query: 5.0 (configured num_results=5).
- Text is disabled: cheaper baseline. Highlights should usually be enough for triage.
- Summary is disabled (recommended for cost-sensitive baseline testing).


,query,top_title,top_url,top_preview,linkedin_present,relevance_keywords_present,cache_hit,actual_cost_usd,est_cost_usd
0,forensic engineer wind damage expert witness F...,David Cowheard | Forensic & Dispute Services P...,https://linkedin.com/in/david-cowheard-a14b2314,witness testimony in the areas of insurance c...,True,True,True,0.01,0.01
1,building envelope consultant moisture intrusio...,Don Williams | Principal at Moisture Intrusion...,https://linkedin.com/in/don-williams-94272611,clients in preventing future water intrusion ...,True,True,True,0.01,0.01
2,forensic accountant business interruption insu...,John W. Foley | Forensic Accountant & Expert W...,https://linkedin.com/in/john-w-foley-31038571,"a qualified expert witness, I’ve testified or...",True,True,True,0.01,0.01
3,meteorologist hurricane wind field expert witn...,Craig Setzer | Chief Meteorologist Royal Carib...,https://linkedin.com/in/craig-setzer-ccm-03095...,igation experience in matters related to meteo...,True,True,True,0.01,0.01
4,fire origin and cause investigator expert witn...,Michael Whedon | Fire and Explosion Investigator,https://linkedin.com/in/michael-whedon-85a639b5,"to insurance companies, private industry and ...",True,True,False,0.01,0.01
5,policyholder attorney bad faith property insur...,Corey Harris | Attorney at Merlin Law Group,https://linkedin.com/in/corey-harris-345b7981,trial attorney exclusively representing polic...,True,True,True,0.01,0.01
6,Texas hail damage property insurance attorney ...,David Rocha | Claims Examiner,https://linkedin.com/in/david-rocha-13997217,"settlement of wind and hail property losses, ...",True,True,False,0.01,0.01
7,insurance appraisal umpire property claims Flo...,"Michael Ruskin | Umpire, Insurance Appraiser, ...",https://linkedin.com/in/michael-ruskin-09214115,". Offices located in Florida, North Carolina, ...",True,True,False,0.01,0.01
8,licensed public adjuster large loss hurricane ...,Frank Altieri | The Policyholders’ Adjuster,https://linkedin.com/in/frank-altieri-5b7bb743,Altieri Insurance Consultants is a nationwide ...,True,True,False,0.01,0.01
9,Xactimate trainer estimator large loss consult...,Weston Faulkner | Large Loss Adjuster / Estima...,https://linkedin.com/in/weston-faulkner-6b4b5a1a,knowledge in Xactimate Estimating Software. T...,True,True,False,0.01,0.01


In [19]:
# Cell 8 - Cost estimate + projections

observed_avg_uncached = float(summary.get("avg_cost_per_uncached_query", 0.0))
observed_spent = float(summary.get("spent_usd", 0.0))

# If a full rerun is cached (avg becomes 0), fall back to a config-based estimate so projections stay useful.
projection_basis = "observed_avg_uncached"
projection_unit_cost = observed_avg_uncached
if projection_unit_cost <= 0:
    baseline_payload = build_exa_payload("baseline projection query", num_results=CONFIG["num_results"])
    projection_unit_cost = _estimate_cost_from_pricing(baseline_payload, int(baseline_payload["numResults"]))
    projection_basis = "estimated_from_current_config"

projections = {
    "projection_basis": projection_basis,
    "unit_cost_usd": round(projection_unit_cost, 6),
    "projected_100_queries_usd": round(projection_unit_cost * 100, 4),
    "projected_1000_queries_usd": round(projection_unit_cost * 1000, 4),
    "projected_10000_queries_usd": round(projection_unit_cost * 10000, 4),
}

print(f"Spent so far (USD): ${observed_spent:.4f}")
print(f"Projection basis: {projection_basis}")
print(f"Projected cost for 100 queries:   ${projections['projected_100_queries_usd']:.4f}")
print(f"Projected cost for 1,000 queries: ${projections['projected_1000_queries_usd']:.4f}")
print(f"Projected cost for 10,000 queries:${projections['projected_10000_queries_usd']:.4f}")

cost_projection_table = pd.DataFrame([projections])
cost_projection_table


Spent so far (USD): $0.8050
Projection basis: observed_avg_uncached
Projected cost for 100 queries:   $1.0592
Projected cost for 1,000 queries: $10.5920
Projected cost for 10,000 queries:$105.9200


,projection_basis,unit_cost_usd,projected_100_queries_usd,projected_1000_queries_usd,projected_10000_queries_usd
0,observed_avg_uncached,0.010592,1.0592,10.592,105.92


In [20]:
# Cell 9 — Decision rubric + PDS integration recommendation (two-pass workflow)

import json
from datetime import datetime

# --- Inputs: try to use whichever dataframes exist in your notebook run ---
# Baseline (people search batch) is usually df_batch
# Credibility pass (picked non-LinkedIn) is usually df_cred

def _safe_mean(series, default=0.0):
    try:
        if series is None or len(series) == 0:
            return default
        return float(series.mean())
    except Exception:
        return default

def _df_exists(name: str) -> bool:
    return name in globals() and globals()[name] is not None

summary = spend_so_far()
df_ledger = ledger_summary()

baseline_df = globals().get("df_batch") if _df_exists("df_batch") else None
cred_df = globals().get("df_cred") if _df_exists("df_cred") else None

# --- Baseline metrics (People discovery) ---
baseline_relevance_rate = None
baseline_linkedin_rate = None
baseline_queries = None

if baseline_df is not None and len(baseline_df) > 0:
    if "relevance_keywords_present" in baseline_df.columns:
        baseline_relevance_rate = _safe_mean(baseline_df["relevance_keywords_present"].astype(int))
    if "linkedin_present" in baseline_df.columns:
        baseline_linkedin_rate = _safe_mean(baseline_df["linkedin_present"].astype(int))
    if "query" in baseline_df.columns:
        baseline_queries = baseline_df["query"].tolist()

# --- Credibility metrics (General search + pick-first-non-LinkedIn) ---
cred_picked_non_linkedin_rate = None
cred_top_linkedin_rate = None
cred_queries = None

if cred_df is not None and len(cred_df) > 0:
    if "picked_is_linkedin" in cred_df.columns:
        cred_picked_non_linkedin_rate = 1.0 - _safe_mean(cred_df["picked_is_linkedin"].astype(int))
    if "top_is_linkedin" in cred_df.columns:
        cred_top_linkedin_rate = _safe_mean(cred_df["top_is_linkedin"].astype(int))
    if "query" in cred_df.columns:
        cred_queries = cred_df["query"].tolist()

# --- Cost metrics ---
unit_cost = float(summary.get("avg_cost_per_uncached_query", 0.0))
spent = float(summary.get("spent_usd", 0.0))
uncached = int(summary.get("uncached_calls", 0))
cache_hits = int(summary.get("cache_hits", 0))
req_count = int(summary.get("request_count", 0))

# Projections from observed unit cost
projections = {
    "cost_per_uncached_query_usd": round(unit_cost, 6),
    "projected_100_queries_usd": round(unit_cost * 100, 4),
    "projected_1_000_queries_usd": round(unit_cost * 1_000, 4),
    "projected_10_000_queries_usd": round(unit_cost * 10_000, 4),
}

# --- Decision rubric (quick + brutal) ---
# We’ll interpret your evidence:
# - Discovery should be reliably relevant (baseline relevance rate high)
# - Credibility pass should produce non-LinkedIn sources deterministically (picked_is_linkedin should be ~0)
# - Cost should be predictable and low

rubric = {
    "Discovery relevance (People Search)": {
        "target": ">= 0.70 relevance signal rate",
        "observed": baseline_relevance_rate,
        "pass": (baseline_relevance_rate is not None and baseline_relevance_rate >= 0.70),
    },
    "Credibility sources (non-LinkedIn evidence)": {
        "target": ">= 0.90 picked non-LinkedIn rate",
        "observed": cred_picked_non_linkedin_rate,
        "pass": (cred_picked_non_linkedin_rate is not None and cred_picked_non_linkedin_rate >= 0.90),
    },
    "Cost per query": {
        "target": "<= $0.02 per uncached query (baseline)",
        "observed": unit_cost,
        "pass": unit_cost <= 0.02,
    },
    "Caching effectiveness": {
        "target": "Cache hits > 0 during iteration",
        "observed": cache_hits,
        "pass": cache_hits > 0,
    },
    "Safety posture": {
        "target": "Public/professional only; redaction on; no text/summaries by default",
        "observed": {
            "moderation": CONFIG.get("moderation"),
            "redact_emails_phones": CONFIG.get("redact_emails_phones"),
            "use_text": CONFIG.get("use_text"),
            "use_summary": CONFIG.get("use_summary"),
        },
        "pass": bool(CONFIG.get("moderation")) and bool(CONFIG.get("redact_emails_phones")) and (CONFIG.get("use_text") is False) and (CONFIG.get("use_summary") is False),
    },
}

passes = sum(1 for k, v in rubric.items() if v["pass"])
total = len(rubric)

headline = "Integrate only for specific workflows"
if passes >= 4:
    headline = "Integrate (two-pass: discovery + credibility verification)"
if passes <= 2:
    headline = "Don't integrate (current settings)"

# --- Recommended integration points in Policy-Dispute-AI-Assistant ---
integration_points = [
    {
        "feature": "Build the Expert Team (CAT-loss bench)",
        "when_to_trigger": "User clicks 'Build Expert Team' or selects dispute type (roof/engineering/BI/appraisal).",
        "pass_1_people_discovery": "Use People Search to shortlist candidates by role (Engineer / Meteorologist / Envelope / Forensic Accountant / Umpire).",
        "pass_2_credibility": "Run general search; pick first non-LinkedIn evidence link per role/person (firm bio, org profile, publications).",
        "output": "Shortlist table + evidence URLs + snippets + cost summary."
    },
    {
        "feature": "Rebut the Report (denial narrative matching)",
        "when_to_trigger": "Denial mentions wear & tear / installation defects / no storm-created opening / foot traffic / thermal expansion.",
        "pass_1_people_discovery": "Find rebuttal-capable experts.",
        "pass_2_credibility": "Attach non-LinkedIn bios/publications to support why they’re relevant.",
        "output": "Rebuttal expert candidates + evidence sources."
    },
    {
        "feature": "Appraisal Path (umpire/appraiser discovery)",
        "when_to_trigger": "User selects appraisal workflow or denial references appraisal/umpire.",
        "pass_1_people_discovery": "Find appraisers/umpires + PAs with appraisal experience.",
        "pass_2_credibility": "Attach firm pages / credential pages.",
        "output": "Appraisal bench + evidence sources."
    },
]

# --- “How to productize” guardrails ---
guardrails = [
    "Never request or store personal addresses. Keep redaction on for emails/phones.",
    "Default to highlights-only (no text/summaries) to keep cost predictable.",
    "Use caching for development + repeat queries; keep a strict budget cap in eval mode.",
    "Require human confirmation before recommending any person as an 'expert' candidate.",
]

# --- Final recommendation object ---
recommendation = {
    "generated_at": datetime.now().isoformat(),
    "headline_recommendation": headline,
    "rubric_passed": f"{passes}/{total}",
    "rubric": rubric,
    "cost_summary": {
        "spent_usd": round(spent, 4),
        "request_count": req_count,
        "uncached_calls": uncached,
        "cache_hits": cache_hits,
        "avg_cost_per_uncached_query_usd": round(unit_cost, 6),
        "projections": projections,
    },
    "observations": {
        "baseline_people_discovery_relevance_rate": baseline_relevance_rate,
        "baseline_linkedin_presence_rate": baseline_linkedin_rate,
        "credibility_picked_non_linkedin_rate": cred_picked_non_linkedin_rate,
        "credibility_top_linkedin_rate": cred_top_linkedin_rate,
        "note": "Discovery can be LinkedIn-heavy; credibility pass ensures non-LinkedIn evidence links for attorney trust."
    },
    "recommended_integration_points": integration_points,
    "guardrails": guardrails,
    "next_improvements": [
        "Add a run_id column to ledger so metrics can be filtered per experiment run.",
        "Add domain quality labels (firm_site / org / directory / article) to credibility results.",
        "Add a small denylist for obvious content-farm domains as they appear.",
        "Add a one-click export bundle (CSV + markdown summary) for sharing with partners."
    ],
}

print(json.dumps(recommendation, indent=2))

{
  "generated_at": "2026-02-23T20:03:48.696379",
  "headline_recommendation": "Integrate (two-pass: discovery + credibility verification)",
  "rubric_passed": "5/5",
  "rubric": {
    "Discovery relevance (People Search)": {
      "target": ">= 0.70 relevance signal rate",
      "observed": 1.0,
      "pass": true
    },
    "Credibility sources (non-LinkedIn evidence)": {
      "target": ">= 0.90 picked non-LinkedIn rate",
      "observed": 1.0,
      "pass": true
    },
    "Cost per query": {
      "target": "<= $0.02 per uncached query (baseline)",
      "observed": 0.010592,
      "pass": true
    },
    "Caching effectiveness": {
      "target": "Cache hits > 0 during iteration",
      "observed": 21,
      "pass": true
    },
    "Safety posture": {
      "target": "Public/professional only; redaction on; no text/summaries by default",
      "observed": {
        "moderation": true,
        "redact_emails_phones": true,
        "use_text": false,
        "use_summary": false
  